In [2]:
!pip install langchain
!pip install pypdf
!pip install openai
!pip install tiktoken
!pip install faiss-cpu

DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!pip install langchain_openai

DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [10]:
OPENAI_API_KEY='sk-proj-E4MrfPzqalVAjicPncXCT3BlbkFJt0yTk2NuijmyhbESyB1z'

In [4]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-proj-E4MrfPzqalVAjicPncXCT3BlbkFJt0yTk2NuijmyhbESyB1z'
 
from langchain_openai import ChatOpenAI
 
# model
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
 
# chain
llm.invoke("태양계 행성의 종류는?")

AIMessage(content='태양계 행성은 수성, 금성, 지구, 화성, 목성, 토성, 천왕성, 해왕성 등으로 구성되어 있습니다. 이 중에서 수성부터 해왕성까지 8개의 행성이 있습니다. 추가로 소행성, 위성, 왜행성 등 다양한 천체들도 태양계에 존재합니다.', response_metadata={'token_usage': {'completion_tokens': 114, 'prompt_tokens': 20, 'total_tokens': 134}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-357490c8-eef4-43b1-a90f-309263125645-0')

In [5]:
import os
import langchain
cwd = os.getcwd()

from os.path import isfile, join
files = [f for f in os.listdir(cwd) if isfile(join(cwd, f)) and f.endswith('.pdf')]
# 현재 디렉토리에 있는 모든 pdf파일의 이름을 리스트로 가져옵니다.

from langchain.document_loaders import PyPDFLoader
pages = []
for f in files:
    loader = PyPDFLoader(join(cwd, f))
    pages = loader.load_and_split()
# 모든 pdf파일을 로드하여 페이지별로 나누고 pages라는 변수명의 리스트로 저장합니다.
# 본 예시에서는 각 페이지별로 문서를 분리하였기때문에 청킹 과정을 생략하였으나
# 읽어온 문서가 하나의 거대한 텍스트 데이터(변수명 data)인 경우 다음과 같은 방법으로
# 청킹할 수 있습니다.
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# pages = text_splitter.split_documents(빈 문서 1.pdf)

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key = 'sk-proj-E4MrfPzqalVAjicPncXCT3BlbkFJt0yTk2NuijmyhbESyB1z')
# OpenAIEmbeddings를 활용해 각 페이지별 embedding을 생성합니다.

from langchain.vectorstores import FAISS
db = FAISS.from_documents(pages, embeddings)

C:\Users\user\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
query = "2024년 6월 중으로 배송하려고 하는데 포워딩 업체 추천해주세요"
# 위와 같이 챗봇에게 질문해보겠습니다.

retrieved_pages = db.similarity_search(query, k=4)
retrieved_contents = " ".join([p.page_content for p in retrieved_pages])
#db로부터 유사성 검색을 실시하여 retrieved_pages를 구하고 해당 페이지의 텍스트를 모두 가져옵니다.

In [8]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)
#gpt-3.5-터보 모델을 사용하겠습니다. (gpt4는 아직 신청대기중이라 ㅠㅜ)


from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_template="""
You are a helpful assistant that that can answer questions about Korean labor laws
based on the following document:{docs}
Only use the factual information from the document to answer the question.
If you feel like you don't have enough information to answer the question,
say "질문에 대한 답을 찾지 못했습니다.".
Your answers should be verbose and detailed.
"""
# 시스템 메시지 프롬프트를 만들기 위한 템플릿입니다. {docs} 자리에는 앞서 검색해서 찾은
# 페이지 컨텐츠가 들어가게 됩니다.
# 시스템 메시지 프롬프트는 챗봇에게 역할을 부여하고 규칙을 설명하는 등의 용도로 사용됩니다.
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)


human_template = "Answer the following question: {question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
# 사람이 질문하는 내용을 담은 프롬프트입니다.

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)
# 시스템과 휴먼 메시지 프롬프트를 결합하여 챗 프롬프트를 만듭니다.

C:\Users\user\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [9]:
from langchain.chains import LLMChain
chain = LLMChain(llm=chat, prompt=chat_prompt)

response = chain.run(question=query, docs=retrieved_contents)
response = response.replace("\n", "")

print(response)
print('답변에 만족하셨나요? Y/N')

NameError: name 'chat' is not defined